In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
import warnings 
warnings.filterwarnings('ignore')

In [3]:
from geopy.geocoders import Nominatim

In [8]:
geolocator = Nominatim(user_agent="Anzy")
location = geolocator.geocode("Milimani Law courts")

print(location.point)
print(location.address)

1 17m 31.6878s S, 36 48m 51.4857s E
Milimani Law Courts, Upperhill Close, Upper Hill, Nairobi, 40476, Kenya


In [9]:
point = location.point
print("Latitude:", point.latitude)
print("Longitude:", point.longitude)

Latitude: -1.2921355
Longitude: 36.8143015941684


In [10]:
universities = pd.read_csv(r"C:\Users\mugok\Desktop\Data sets\top_universities.csv")
universities.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


In [13]:
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None

universities[['Latitude', 'Longitude']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(universities["Latitude"])) / len(universities)) * 100))

# Drop universities that were not successfully geocoded
universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(
    universities, geometry=gpd.points_from_xy(universities.Longitude, universities.Latitude))
universities.crs = {'init': 'epsg:4326'}

97.8021978021978% of addresses were geocoded!


In [23]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Create a map
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m)

 #Display the map
embed_map(m,'map.html')